In [1]:
import sys
sys.path.append("..")
import os
from os.path import join, dirname
from dotenv import load_dotenv
from pathlib import Path
import pandas as pd
import numpy as np

import utils.top_n_box as top_n_box
import utils.read_data as rd

In [2]:
load_dotenv(verbose=True)
dotenv_path = join(Path().resolve(), '.env')
load_dotenv(dotenv_path)

True

In [3]:
GOOGLE_DRIVE_PATH = os.environ.get("GOOGLE_DRIVE_PATH")
DATA_PATH = GOOGLE_DRIVE_PATH + '/train_data'

## サンプルデータを準備

In [4]:
df = rd.read_horse_race_csv(DATA_PATH)

horse_race_20201128.csv


c:\users\endo_\anaconda3\envs\py37\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (17,65,113,157,161,205,209,253,257) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 639818 entries, 0 to 639817
Columns: 288 entries, race_id to rentan3-5
dtypes: float64(89), int64(13), object(186)
memory usage: 1.4+ GB


In [6]:
df.head()

,race_id,rank,frame_number,horse_number,horse_id,sex_and_age,burden_weight,rider_id,goal_time,goal_time_dif,...,hukusyo_second-5,hukusyo_third-5,wakuren-5,umaren-5,wide_1_2-5,wide_1_3-5,wide_2_3-5,umatan-5,renhuku3-5,rentan3-5
0,2.017091e+11,11,5,8,2015100928,牝2,51.0,1168,1:11.2,1.1/4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.017091e+11,9,6,11,2015100928,牝2,54.0,1126,1:14.9,アタマ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.018080e+11,12,2,3,2015100928,牝3,54.0,732,1:30.8,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.018080e+11,4,2,4,2015100928,牝3,54.0,1154,1:14.4,1.3/4,...,260,190,"6,240","17,470","4,710","2,280","1,030","37,440","22,660","303,790"
4,2.018090e+11,16,6,12,2015100928,牝3,54.0,732,1:14.1,クビ,...,170,210,"2,680","2,560",740,"1,170",500,"6,700","3,970","32,740"


In [7]:
for c in df.columns:
    if "tan" in c or "huku" in c or "wide" in c:
        print(c)

tansyo
hukusyo_first
hukusyo_second
hukusyo_third
wide_1_2
wide_1_3
wide_2_3
umatan
renhuku3
rentan3
tansyo-1
hukusyo_first-1
hukusyo_second-1
hukusyo_third-1
wide_1_2-1
wide_1_3-1
wide_2_3-1
umatan-1
renhuku3-1
rentan3-1
tansyo-2
hukusyo_first-2
hukusyo_second-2
hukusyo_third-2
wide_1_2-2
wide_1_3-2
wide_2_3-2
umatan-2
renhuku3-2
rentan3-2
tansyo-3
hukusyo_first-3
hukusyo_second-3
hukusyo_third-3
wide_1_2-3
wide_1_3-3
wide_2_3-3
umatan-3
renhuku3-3
rentan3-3
tansyo-4
hukusyo_first-4
hukusyo_second-4
hukusyo_third-4
wide_1_2-4
wide_1_3-4
wide_2_3-4
umatan-4
renhuku3-4
rentan3-4
tansyo-5
hukusyo_first-5
hukusyo_second-5
hukusyo_third-5
wide_1_2-5
wide_1_3-5
wide_2_3-5
umatan-5
renhuku3-5
rentan3-5


In [8]:
test_df = df[["race_id", "rank", "tansyo", "hukusyo_first", "hukusyo_second", "hukusyo_third", "wide_1_2", "wide_1_3", "wide_2_3"]]
test_df.head()

,race_id,rank,tansyo,hukusyo_first,hukusyo_second,hukusyo_third,wide_1_2,wide_1_3,wide_2_3
0,2.017091e+11,11,"4,780",830,260,190,"4,710","2,280","1,030"
1,2.017091e+11,9,"2,630",740,480,510,"7,900","6,830","4,080"
2,2.018080e+11,12,450,170,110,380,260,"1,260",650
3,2.018080e+11,4,"2,730",800,510,190,"3,700","2,050","1,400"
4,2.018090e+11,16,"1,420",450,930,310,"5,690","1,750","3,370"


In [9]:
test_race_ids = list(set(test_df["race_id"].values))[:4]
print(test_race_ids)

[202005020701.0, 202005020702.0, 202005020703.0, 202005020704.0]


In [10]:
def extract_race(horse_race_df, race_id):
    return horse_race_df[horse_race_df["race_id"] == race_id]

In [11]:
top_n_test_df = test_df[test_df["race_id"] == test_race_ids[0]]
for race_id in test_race_ids[1:]:
    top_n_test_df = top_n_test_df.append(extract_race(test_df, race_id))

In [12]:
print(test_race_ids[0])
extract_race(top_n_test_df, test_race_ids[0])

202005020701.0


,race_id,rank,tansyo,hukusyo_first,hukusyo_second,hukusyo_third,wide_1_2,wide_1_3,wide_2_3
152297,2.020050e+11,9,760,160,160,110,440,240,250
157393,2.020050e+11,3,760,160,160,110,440,240,250
161518,2.020050e+11,6,760,160,160,110,440,240,250
173433,2.020050e+11,13,760,160,160,110,440,240,250
173692,2.020050e+11,7,760,160,160,110,440,240,250
176902,2.020050e+11,14,760,160,160,110,440,240,250
177499,2.020050e+11,15,760,160,160,110,440,240,250
181923,2.020050e+11,11,760,160,160,110,440,240,250
183316,2.020050e+11,2,760,160,160,110,440,240,250
183467,2.020050e+11,12,760,160,160,110,440,240,250


In [13]:
print(test_race_ids[1])
extract_race(top_n_test_df, test_race_ids[1])

202005020702.0


,race_id,rank,tansyo,hukusyo_first,hukusyo_second,hukusyo_third,wide_1_2,wide_1_3,wide_2_3
153390,2.020050e+11,8,"5,960",570,110,110,"1,620","1,490",150
156335,2.020050e+11,12,"5,960",570,110,110,"1,620","1,490",150
160537,2.020050e+11,3,"5,960",570,110,110,"1,620","1,490",150
163708,2.020050e+11,9,"5,960",570,110,110,"1,620","1,490",150
166408,2.020050e+11,15,"5,960",570,110,110,"1,620","1,490",150
169303,2.020050e+11,10,"5,960",570,110,110,"1,620","1,490",150
169359,2.020050e+11,1,"5,960",570,110,110,"1,620","1,490",150
172223,2.020050e+11,6,"5,960",570,110,110,"1,620","1,490",150
178408,2.020050e+11,5,"5,960",570,110,110,"1,620","1,490",150
180110,2.020050e+11,4,"5,960",570,110,110,"1,620","1,490",150


In [14]:
print(test_race_ids[2])
extract_race(top_n_test_df, test_race_ids[2])

202005020703.0


,race_id,rank,tansyo,hukusyo_first,hukusyo_second,hukusyo_third,wide_1_2,wide_1_3,wide_2_3
150181,2.020050e+11,2,200,110,150,170,280,330,540
151642,2.020050e+11,1,200,110,150,170,280,330,540
152391,2.020050e+11,16,200,110,150,170,280,330,540
154900,2.020050e+11,12,200,110,150,170,280,330,540
157431,2.020050e+11,5,200,110,150,170,280,330,540
162444,2.020050e+11,6,200,110,150,170,280,330,540
166913,2.020050e+11,7,200,110,150,170,280,330,540
173980,2.020050e+11,10,200,110,150,170,280,330,540
174701,2.020050e+11,13,200,110,150,170,280,330,540
174804,2.020050e+11,8,200,110,150,170,280,330,540


In [15]:
print(test_race_ids[3])
extract_race(top_n_test_df, test_race_ids[3])

202005020704.0


,race_id,rank,tansyo,hukusyo_first,hukusyo_second,hukusyo_third,wide_1_2,wide_1_3,wide_2_3
166105,2.020050e+11,12,610,220,260,"2,200",770,"8,930","9,980"
169687,2.020050e+11,1,610,220,260,"2,200",770,"8,930","9,980"
172982,2.020050e+11,8,610,220,260,"2,200",770,"8,930","9,980"
173348,2.020050e+11,4,610,220,260,"2,200",770,"8,930","9,980"
175314,2.020050e+11,11,610,220,260,"2,200",770,"8,930","9,980"
176653,2.020050e+11,10,610,220,260,"2,200",770,"8,930","9,980"
178709,2.020050e+11,5,610,220,260,"2,200",770,"8,930","9,980"
183839,2.020050e+11,7,610,220,260,"2,200",770,"8,930","9,980"
187811,2.020050e+11,2,610,220,260,"2,200",770,"8,930","9,980"
188722,2.020050e+11,6,610,220,260,"2,200",770,"8,930","9,980"


In [16]:
top_n_test_df["rank"].replace("中", -1, inplace=True)
top_n_test_df = top_n_test_df.astype({"rank": np.int32})

In [17]:
import random 
# 各レースの疑似予測データを作成
race1_df = extract_race(top_n_test_df, test_race_ids[0])
predicted_ranks_df = pd.DataFrame({
    "race_id": test_race_ids[0], 
    "rank": [int(r) for r in race1_df ["rank"].values]
})

for race_id in test_race_ids[1:]:
    extracted_race_df = extract_race(top_n_test_df, race_id)
    predicted_ranks = random.sample([i + 1
        for i in range(len(extracted_race_df))
    ], len(extracted_race_df))
    predicted_ranks_df = predicted_ranks_df.append(pd.DataFrame(
        {"race_id": race_id, "rank": predicted_ranks}
    ))
predicted_ranks_df.head()

,race_id,rank
0,2.020050e+11,9
1,2.020050e+11,3
2,2.020050e+11,6
3,2.020050e+11,13
4,2.020050e+11,7


In [18]:
top_n_test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61 entries, 152297 to 193381
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   race_id         61 non-null     float64
 1   rank            61 non-null     int32  
 2   tansyo          61 non-null     object 
 3   hukusyo_first   61 non-null     object 
 4   hukusyo_second  61 non-null     object 
 5   hukusyo_third   61 non-null     object 
 6   wide_1_2        61 non-null     object 
 7   wide_1_3        61 non-null     object 
 8   wide_2_3        61 non-null     object 
dtypes: float64(1), int32(1), object(7)
memory usage: 4.5+ KB


## リターン計算例

In [19]:
from utils.top_n_box import TopNBox
import utils.return_calculation as return_calculation

In [20]:
print(TopNBox.ticket_types)

['tansho', 'fukusho', 'wakuren', 'umaren', 'umatan', 'wide', 'sanrempuku', 'sanrentan']


In [23]:
N = 3
PRICE_OF_BETTING_TICKET = 100
hit_num = 0
ret = []
ret_rates = []
ticket_num = 0

for race_id in test_race_ids:
    extracted_race_df = extract_race(top_n_test_df, race_id)
    target_ranks = extracted_race_df["rank"].values
    prizes = np.array([int("".join((v.split(",")))) for v in extracted_race_df["tansyo"].values])
    predicted_ranks = extract_race(predicted_ranks_df, race_id)["rank"].values
    
    top_n_box = TopNBox(predicted_ranks, N)
    tickets = top_n_box.output_tickets("tansho")
    n, r = return_calculation.tansho(tickets, target_ranks, prizes)
    hit_num += n
    ticket_num += len(tickets)
    ret.append(r)
    ret_rates.append(r/(ticket_num*PRICE_OF_BETTING_TICKET))
    
    print(f"----{race_id}----")
    print(prizes)
    print("predicted_ranks:", predicted_ranks)
    print("target_ranks:", target_ranks)
    print("tickets:", tickets)
    print("ticket_num:", ticket_num)
    print("hit_num:", n)
    print("return: ", r)
    
print("")
print("-----tansyo resulut-----")
print("total tickets:", ticket_num)
print("total hit:", hit_num)
print("total return:", np.sum(ret))
print("hit rate:", hit_num/ticket_num)
print("return rate:", np.sum(ret)/(ticket_num*100))
print("return rate std:", np.std(ret_rates, ddof=1))

760
760
----202005020701.0----
[760 760 760 760 760 760 760 760 760 760 760 760 760 760 760 760]
predicted_ranks: [ 9  3  6 13  7 14 15 11  2 12 -1 10  5  8  1  4]
target_ranks: [ 9  3  6 13  7 14 15 11  2 12 -1 10  5  8  1  4]
tickets: [array([14], dtype=int64), array([8], dtype=int64), array([1], dtype=int64)]
ticket_num: 3
hit_num: 1
return:  760
0
----202005020702.0----
[5960 5960 5960 5960 5960 5960 5960 5960 5960 5960 5960 5960 5960 5960
 5960 5960]
predicted_ranks: [ 5  3  8  4  9 15 12 10 16  1  2 13  6 14  7 11]
target_ranks: [ 8 12  3  9 15 10  1  6  5  4 16  2 13  7 11 14]
tickets: [array([9], dtype=int64), array([10], dtype=int64), array([1], dtype=int64)]
ticket_num: 6
hit_num: 0
return:  0
200
200
----202005020703.0----
[200 200 200 200 200 200 200 200 200 200 200 200 200 200 200 200]
predicted_ranks: [ 3  1  5 16  9  8  6 15 12  7  4  2 10 13 14 11]
target_ranks: [ 2  1 16 12  5  6  7 10 13  8  4 11 14  9  3 15]
tickets: [array([1], dtype=int64), array([11], dtype=int64)